In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!ls "gdrive/MyDrive/Neuromatch Project"

 Animation	 'Group 2 : Abstract.gdoc'  'Project Abstract.gdoc'
'Colab Drafts'	  LICENSE		     README.md
'Colab Project'   LINKS.gdoc		     Template_ECoG_memory_nback
 Datasets	 'Metadata Notes.gdoc'	    'Untitled spreadsheet.gsheet'


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# @title Data retrieval
import os, requests

path = "./gdrive/MyDrive/Neuromatch Project/Datasets/"
name = 'memory_nback.npz'
fname = path+name
url = "https://osf.io/xfc7e/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [5]:
# @title Data loading
import numpy as np

alldat = np.load(fname, allow_pickle=True)['dat']

# Select just one of the recordings here. This is subject 1, block 1.
dat = alldat[1][1]

print(dat.keys())

dict_keys(['V', 't_off', 'locs', 'srate', 'scale_uv', 't_on', 'target', 'stim_id', 'response', 'rt', 'expinfo', 'hemisphere', 'lobe', 'gyrus', 'Brodmann_Area'])


In [6]:
# compute spectral power above 50Hz and low-pass below 10Hz
# power is always positive, so we normalize it by its average

from scipy import signal
def voltage_from_data(complete_dataset = alldat, subject = 1, experiment = 1, upper_limit = 50, lower_limit = 10):

  # pick subject 1 and experiment 1
  dat = complete_dataset[subject][experiment]
  V = dat['V'].astype('float32') # always convert the voltage data to float32!

  # if np.isnan(lower_limit) == False and np.isnan(upper_limit) == False :
  #   b, a = signal.butter(3, [lower_limit], btype='high', fs=1000)


  # high-pass filter above `lower-limit` Hz
  if np.isnan(lower_limit) == False : 
    b, a = signal.butter(3, [lower_limit], btype='high', fs=1000)
    V = signal.filtfilt(b, a, V, 0)

  # low-pass filter above `upper-limit` Hz
  # if np.isnan(upper_limit) == False :
  else :
    b, a = signal.butter(3, [upper_limit], btype='low', fs=1000)
    V = signal.filtfilt(b, a, V, 0)

  # compute smooth envelope of this signal = approx power
  V = np.abs(V)**2
  b, a = signal.butter(3, [10], btype='low', fs=1000)
  V = signal.filtfilt(b, a, V, 0)

  # compute smooth envelope of this signal = approx power
  # normalize each channel so its mean power is 1
  V = V/V.mean(0)

  return V


def get_epochs(complete_dataset = alldat, subject = 1, experiment = 1, upper_limit = 50, lower_limit = 10, time_window = [-400, 1600], per_trial = True):
  V = voltage_from_data(alldat, subject, experiment, upper_limit, lower_limit)
  dat = complete_dataset[subject, experiment]

  # divide into trials and average
  nt, nchan = V.shape
  nstim = len(dat['t_on'])

  # Use a timerange from time_wondow[0] to time_window[1] after the stimulus onset

  trange = np.arange(time_window[0], time_window[1])
  calc_trange = time_window[1] - time_window[0]
  ts = dat['t_on'][:, np.newaxis] + trange
  if per_trial : 
    V_epochs = np.reshape(V[ts, :], (nstim, calc_trange, nchan))
  else :
    V_epochs = np.reshape(V[ts, :], (nstim * calc_trange, nchan))

  return V_epochs

def get_resp_base(mean = False, complete_dataset = alldat, subject = 1, experiment = 1):
  V = voltage_from_data(complete_dataset, subject, experiment)
  dat = complete_dataset[subject][experiment]

    # divide into trials and average
  nt, nchan = V.shape
  nstim = len(dat['t_on'])

  # use a timerange from 400ms before to 1600ms after the stimulus onset

  trange = np.arange(-400, 1600)
  ts = dat['t_on'][:, np.newaxis] + trange
  V_epochs = np.reshape(V[ts, :], (nstim, 2000, nchan))

  if mean :
    V_resp = (V_epochs[dat['response'] == 1]).mean(0)
    V_base = (V_epochs[dat['response'] == 0]).mean(0)
    return V_resp, V_base, trange

  else :
    V_resp_tri = V_epochs[dat['response'] == 1]
    V_base_tri = V_epochs[dat['response'] == 0]
    return V_resp_tri, V_base_tri, trange




In [7]:
V_resp, V_base, trange = get_resp_base()
V_resp_mean, V_base_mean, trange_mean = get_resp_base(True)

## PCA

In [8]:
def get_variance_explained(evals):
  # Cumulatively sum the eigenvalues
  csum = np.cumsum(evals)

  # Normalize by the sum of eigenvalues
  variance_explained = csum/np.sum(evals)

  return variance_explained

def plot_variance_explained(variance_explained, axis = False):

  plt.figure()
  plt.plot(np.arange(1, len(variance_explained) + 1), variance_explained,
           '--k')
  plt.xlabel('Cummulated Number of P_components from channels')
  plt.ylabel('Variance explained')
  if axis : 
    plt.ylim([0, 1])
  plt.show()

def sort_evals_descending(evals, evectors):

  index = np.flip(np.argsort(evals))
  evals = evals[index]
  evectors = evectors[:, index]
  if evals.shape[0] == 2:
    if np.arccos(np.matmul(evectors[:, 0],1 / np.sqrt(2) * np.array([1, 1]))) > np.pi / 2:
      evectors[:, 0] = -evectors[:, 0]
    if np.arccos(np.matmul(evectors[:, 1],1 / np.sqrt(2)*np.array([-1, 1]))) > np.pi / 2:
      evectors[:, 1] = -evectors[:, 1]

  return evals, evectors

def plot_PCA_weights(weights):

  vmax = np.max(np.abs(weights))

  fig, ax = plt.subplots()
  cmap = plt.cm.get_cmap('seismic')
  plt.imshow(np.real(weights), cmap=cmap)
  plt.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False)
  plt.clim(-vmax, vmax)
  plt.colorbar(ticks=np.arange(-vmax, vmax, 0.1))
  ax.set_xticks([])
  ax.set_yticks([])
  plt.show()

def plot_V_activity(activity):

  # vmax = np.max(activity)
  # vmin = np.min(activity)
  vmax = 3
  vmin = 0

  fig, ax = plt.subplots()
  cmap = plt.cm.get_cmap('plasma')
  plt.imshow(np.real(activity), cmap=cmap)
  plt.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False)
  plt.clim(vmin, vmax)
  plt.colorbar(ticks=np.arange(vmin, vmax, 0.1))
  ax.set_xticks([])
  ax.set_yticks([])
  plt.show()

In [9]:
def pca(X):
  
  X_mean = X - np.mean(X, 0)
  X_COV = np.cov(X_mean.T)

  evals, evectors = np.linalg.eigh(X_COV)

  evals, evectors = sort_evals_descending(evals, evectors)

  score = X @ evectors

  # Calculate the variance explained
  variance_explained = get_variance_explained(evals)

  return variance_explained, score, evals, evectors

In [10]:
# Calculate RMS of voltages
def RMS(time_series):
  return np.sqrt(np.mean((np.square(time_series))))

In [17]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import random

In [18]:
test_size = 0.2                    # Value between 0 and 1
version_1_or_2 = False             # True is version 1 with only channels as columns, Flase is version 2 with channels and time steps in columns
pca_reduction = True # False               # Are channels reduced by number of pca components?
data_processing = ''            # string describing the data procesing procedure before modelling, 'rms', 'derivative', 'raw'
trials_are_meaned = False          # Are the trials meaned?
num_of_principal_comp = 20         # If pca_reduction, choose how many principal components are kept    
frequency_range = (50, np.nan)     # Chooosing the frequencies kept
time_window = [-400, 1600]         # Choosing the time window around the stimulus
time_bin_size = 10                     # Size of the time bin
subject_list = [0, 1]                  # Subject number, has to be in array even if only one subject is wanted
experiment_list = [1, 2, 3]            # Experiment number, has to be in an array if only one subject is wanted

In [19]:
def dataset_generation(version_1_or_2, test_size, data_processing, pca_reduction, num_of_principal_comp, frequency_range, time_window, time_bin_size, subject_list, experiment_list ):

  all_data = []
  all_scores = []
  all_responses = []

  lower_limit, upper_limit = frequency_range

  # Construct/Concat the dataset

  for subject in subject_list : 
    for experiment in experiment_list : 

      # complete_dataset = alldat, subject = 1, experiment = 1, upper_limit = 50, lower_limit = 10'
      epochs = get_epochs(alldat, subject, experiment, upper_limit, lower_limit, time_window, True) # do some more testing
      
      V_epochs = []
      index = 0
      for epoch in epochs : 
        df = pd.DataFrame(epoch)
        
        # Binning
        df = pd.DataFrame(df.groupby(df.index // time_bin_size).mean()) 
            # Keep in mind the downsample, also the function `resample` in pandas but needs to add the dates as index
            # Same function `resample` and `decimate` in scipy could work
        
        if data_processing=='rms':
          df = pd.DataFrame(df.apply(RMS, axis=0)).transpose()

        elif data_processing=='derivative':
          df = df.apply(np.diff, axis=0)

        # Creating index values per trial, ex 
        value = (10000*(subject + 1) + 1000*(experiment+1) + index) 
        df["trial_id"] = value

        # Assigning target values
        df["target"] = alldat[subject][experiment]['response'][index]
        V_epochs.append(df)
        index+=1  
     
      
      V_epochs = pd.concat(V_epochs)

      all_data.append(V_epochs)

  # Stacking all the experiments from all subjects in one
  all_data_df = pd.concat(all_data)
  print("all_data shape : ", all_data_df.shape)

  # Take one value per trial 
  all_data_grouped = all_data_df.groupby(by= ["trial_id"]).tail(1).set_index("trial_id")
  X_train_index, X_test_index, y_train_index, y_test_index = train_test_split(all_data_grouped.drop(["target"], axis = 1), all_data_grouped["target"], test_size=test_size, random_state=42, stratify=all_data_grouped["target"])
    
  train_data = all_data_df[all_data_df["trial_id"].apply(lambda x : x in X_train_index.index.values)].set_index("trial_id")
  X_train = train_data.drop(["target"], axis = 1)
  y_train = train_data["target"]

  test_data = all_data_df[all_data_df["trial_id"].apply(lambda x : x in X_test_index.index.values)].set_index("trial_id")
  X_test = test_data.drop(["target"], axis = 1)
  y_test = test_data["target"]

  # Training and on test sets
  if pca_reduction : 

    # train_extra_col = X_train[extra_col]
    train_PCA = PCA(n_components = num_of_principal_comp).fit(X_train) # To be checked if that is the best procedure
    X_train = pd.DataFrame(train_PCA.transform(X_train), index = X_train.index)
    X_test = pd.DataFrame(train_PCA.transform(X_test), index = X_test.index)

  # 
  if version_1_or_2 == False :

    X_train = X_train.reset_index().groupby(by = ["trial_id"]).apply(lambda g : pd.DataFrame([g.drop(["trial_id"], axis=1).to_numpy().flatten()])).reset_index().drop(["level_1","trial_id"], axis = 1)# numpy to flatten - fastest way to flatten your matrix
    y_train = y_train.reset_index().groupby("trial_id").tail(1).drop(["trial_id"], axis = 1)

    X_test = X_test.reset_index().groupby(by = ["trial_id"]).apply(lambda g : pd.DataFrame([g.drop(["trial_id"], axis=1).to_numpy().flatten()])).reset_index().drop(["level_1", "trial_id"], axis = 1)
    y_test = y_test.reset_index().groupby("trial_id").tail(1).drop(["trial_id"], axis = 1)

  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = dataset_generation(version_1_or_2, test_size, data_processing, pca_reduction, num_of_principal_comp, frequency_range, time_window, time_bin_size, subject_list, experiment_list)

all_data shape :  (120000, 66)


In [20]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((480, 4000), (120, 4000), (480, 1), (120, 1))